# Trabalhando bem

Vamos abordar alguns aspectos de como fazer um algoritmo/pacote
nos padrões esperados do Julia, e também um workflow para desenvolver
um algoritmo rápido, e depois melhorá-lo.

## Um pacote

Uma pacote em Julia, é um diretório com a configuração

 - NomeDoPacote.jl/
   - src/
     - NomeDoPacote.jl
     - Outros arquivos
   - test/
     - runtests.jl
     - Arquivos auxiliares de teste.
   - README.md
   - LICENSE.md
   - .travis.yml
   - Outros arquivos
   
Por mais incomum que seja, o nome do nosso pacote será o nome do diretório,
**incluindo o .jl**. Além disso, esse mesmo nome será o nome do arquivo
principal do pacote.

Os arquivos do `src/` sozinhos não fazem nada. Eles declaram e definem as
funções do pacote. O arquivo `test/` é quem chama o código, assim como
o usuário chamaria. O arquivo `runtests.jl` é o arquivo chamado quando o
usuário fizer `Pkg.test("NomeDoPacote")`.
No índice de pacotes é mostrado se esse teste é bem sucedido.

O arquivo `README.md` é o arquivo principal de informação do seu pacote,
e o arquivo `LICENSE.md` é o arquivo indicando qual a licensa do seu pacote.
A licensa mais utilizada aqui é a MIT, mas outras podem ser utilizadas.

O arquivo `.travis.yml` é um arquivo muito específico que roda os testes
do seu pacote online. Ele exige configuração, e não vamos falar dele agora.

## NomeDoPacote.jl

O arquivo `NomeDoPacote.jl` é o arquivo principal do seu pacote.
A maneira normal de começar é criar um módulo para seu código.

````julia
module NomeDoPacote

...

end
````

Dentro desse módulo, normalmente incluímos códigos auxiliares. Por exemplo, vamos colocar a busca linear num
arquivo separado `buscaLinear.jl`. Então colocamos `include("buscaLinear.jl")`.

Agora criamos a nossa função `metodo_gradiente` e colocamos um comando
`export metodo_gradiente` para que usuários possam usar essa função.

Agora criamos algum teste para nosso código. Vamos criar o arquivo
`teste_quad.jl` contendo testes de funções quadráticas, e o no arquivo
`runtests.jl` colocamos `include("teste_quad.jl")`.

Veja o MetodoGradiente.jl.

## Workflow de um código de otimização

Até bem pouco tempo, tínhamos duas opções de implementações
de algoritmos de otimização

 - C/Fortran: Um algoritmo rápido, competitivo, porém trabalhoso de fazer e usar;
 - MatLab: Um algoritmo fácil de escrever, porém limitado em alguns aspectos de velocidade e liberdade de uso.

O Python criou uma alternativa que não fui suficientemente adotada por não ser fácil o suficiente,
mas com Julia, temos uma oportunidade nova.

O Julia permite uma implementação rápida, é livre, e tem uma velocidade boa.
Além disse, permite uma integração com C e Fortran muito maior, que pode ser feita gradualmente.
Eventualmente, se necessário, pode-se fazer uma migração total.

Esse paradigma funciona para quase qualquer tipo de aplicação em Julia,
mas para otimização vamos ver como o CUTEst pode nos ajudar.1

### Ponto inicial

A primeira preocupação deve ser fazer o algoritmo funcionar.

 - Não se preocupe em deixar o código eficiente;
 - Não se preocupe com sistemas lineares;
 - Não se preocupe com BLAS
 - Não se preocupe com variáveis repetidas;
 - Não se preocupe com matrizes criadas à toa;

Palavras do Donald Knuth:

> Programmers waste enormous amounts of time thinking about, or worrying about, the speed of noncritical parts of their programs, and these attempts at efficiency actually have a strong negative impact when debugging and maintenance are considered. We should forget about small efficiencies, say about 97% of the time: **premature optimization is the root of all evil.** Yet we should not pass up our opportunities in that critical 3%.
> - Structured Programming With Go To Statements", Computing Surveys, Vol 6, No 4, December 1974 

Escreva os testes, decida o que o código precisa resolver e implemente tudo.